In [1]:
cd Triple_view/

/home/gene/PycharmProjects/Machine/Triple_view


In [11]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from triple_data_utils import _concatenated_check_and_labels, _resize_after_concat, _three_view_show, _unpack_views, _single_views_resize
import matplotlib.pyplot as plt
from skimage import transform

In [3]:
both_dir = '/home/gene/PycharmProjects/Machine/CNN/large_files'
NL_dir = 'all_NL'
ABNL_dir = 'all_ABNL'

# load the data
NL_cases, ABNL_cases = _concatenated_check_and_labels(both_dir, NL_dir, ABNL_dir)
# with view expand, total of 298 NL and 298 ABNL cases.  One NL case deleted since it was views of the elbow (?GK_255?)
print(len(NL_cases), len(ABNL_cases))

3 3


In [4]:
NL_three, NL_two, NL_one = NL_cases
NL_three = shuffle(NL_three)
NL_two = shuffle(NL_two)
NL_one = shuffle(NL_one)
print(len(NL_three), len(NL_two), len(NL_one))
ABNL_three, ABNL_two, ABNL_one = ABNL_cases
ABNL_three = shuffle(ABNL_three)
ABNL_two = shuffle(ABNL_two)
ABNL_one = shuffle(ABNL_one)

print(len(ABNL_three), len(ABNL_two), len(ABNL_one))

276 22 0
218 75 5


## Manual separation of validation and test data from 3 views set

In [5]:
val_triple = NL_three[:20] + ABNL_three[:20]
val_y = np.concatenate((np.zeros(len(NL_three[:20])), np.ones(len(ABNL_three[:20]))), axis=0)
test_triple = NL_three[20:40] + ABNL_three[20:40]
test_y = np.concatenate((np.zeros(len(NL_three[20:40])), np.ones(len(ABNL_three[20:40]))), axis=0)
NL_cases_3 = NL_three[40:]
ABNL_cases_3 = ABNL_three[40:]
print('Three views NL:{} ABNL:{}'.format(len(NL_cases_3), len(ABNL_cases_3)))
print(len(val_triple), val_y.shape, len(test_triple), test_y.shape)

Three views NL:236 ABNL:178
40 (40,) 40 (40,)


## Working on training set

In [8]:
all_NL_views = _unpack_views(NL_two) + _unpack_views(NL_cases_3)
all_ABNL_views = _unpack_views(ABNL_two) + _unpack_views(ABNL_one) + _unpack_views(ABNL_cases_3)
tot_y = np.concatenate((np.zeros(len(all_NL_views)), np.ones(len(all_ABNL_views))), axis=0)
all_cases = all_NL_views + all_ABNL_views

In [9]:
print(len(all_ABNL_views), len(all_NL_views))
print(tot_y.shape)

689 752
(1441,)


In [12]:
reshaped_cases = _single_views_resize(all_cases)
print(reshaped_cases.shape)

(1441, 300, 300, 1)


In [13]:
reshaped_cases, tot_y = shuffle(reshaped_cases, tot_y)
np.save('./../CNN/triple_view_data/reshaped_single_views', reshaped_cases)
np.save('./../CNN/triple_view_data/single_views_y', tot_y)

## Work of val and test sets

In [14]:
val_triple, val_y = shuffle(val_triple, val_y)
val_cases = _resize_after_concat(val_triple)
test_triple, test_y = shuffle(test_triple, test_y)
test_cases = _resize_after_concat(test_triple)

In [15]:
print(val_cases.shape, test_cases.shape)
np.save('./../CNN/triple_view_data/val_triple_views', val_cases)
np.save('./../CNN/triple_view_data/val_triple_y', val_y)
np.save('./../CNN/triple_view_data/test_triple_views', test_cases)
np.save('./../CNN/triple_view_data/test_triple_y', test_y)

(40, 3, 300, 300, 1) (40, 3, 300, 300, 1)


## make sure to run code below first since _resize_after_concat modifies original list

In [16]:
val_triple = NL_three[:20] + ABNL_three[:20]
val_y = np.concatenate((np.zeros(len(NL_three[:20])), np.ones(len(NL_three[:20]))), axis=0)
test_triple = NL_three[20:40] + ABNL_three[20:40]
test_y = np.concatenate((np.zeros(len(NL_three[20:40])), np.ones(len(NL_three[20:40]))), axis=0)
NL_cases_3 = NL_three[40:]
ABNL_cases_3 = ABNL_three[40:]
print('Three views NL:{} ABNL:{}'.format(len(NL_cases_3), len(ABNL_cases_3)))
print(len(val_triple), val_y.shape, len(test_triple), test_y.shape)

Three views NL:236 ABNL:178
40 (40,) 40 (40,)


In [18]:
single_view_vals_cases = _single_views_resize(_unpack_views(val_triple))
single_view_vals_y = np.array(([0]*20*3 + [1]*20*3), np.int64)
single_view_test_cases = _single_views_resize(_unpack_views(test_triple))
single_view_test_y = np.array(([0]*20*3 + [1]*20*3), np.int64)

In [20]:
single_view_vals_cases, single_view_vals_y = shuffle(single_view_vals_cases, single_view_vals_y)
print(single_view_vals_cases.shape, single_view_vals_y.shape)
single_view_test_cases, single_view_test_y = shuffle(single_view_test_cases, single_view_test_y)
print(single_view_test_cases.shape, single_view_test_y.shape)

(120, 300, 300, 1) (120,)
(120, 300, 300, 1) (120,)


In [21]:
np.save('./../CNN/triple_view_data/val_single_views', single_view_vals_cases)
np.save('./../CNN/triple_view_data/val_single_y', single_view_vals_y)
np.save('./../CNN/triple_view_data/test_single_views', single_view_test_cases)
np.save('./../CNN/triple_view_data/test_single_y', single_view_test_y)